## Importing Relevant Libraries
#### - Numpy, Pandas, Seaborn, Nltk etc...

In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import nltk.tokenize

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.tokenize import RegexpTokenizer

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Lee Seung
[nltk_data]     Soo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Lee Seung
[nltk_data]     Soo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Read data

In [3]:
fakenewsdataset = pd.read_csv("fake_or_real_news.csv")
fakenewsdataset.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


## Removal of stop words
#### - Punctuation, space, etc...

In [4]:
#Removing stop words
def cleanupDoc(s):
    stopset = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(s)
    cleanup = " ".join(filter(lambda word: word not in stopset, s.split()))
    return cleanup

#Creating new column for number of words in title
def word_count(title):
    count = 1
    for x in title:
        if x == " ":
            count += 1
    return count

fakenewsdataset["title count"] = word_count(fakenewsdataset['title'])
for y in range(0,6335):
    fakenewsdataset["title count"][y] = word_count(fakenewsdataset['title'][y])
fakenewsdataset


#create variable for tokenize
fakenewsdataset["tokenized"] = fakenewsdataset["title count"]
for x in range(0,6335):
    #Removing Stop Words
    y = cleanupDoc(fakenewsdataset["title"][x])
    #Removing Punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    #Tokenizing
    fakenewsdataset["tokenized"][x] = tokenizer.tokenize(y)
    
fakenewsdataset.head()

C:\Users\LEESEU~1\AppData\Local\Temp/ipykernel_20036/3533526121.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fakenewsdataset["title count"][y] = word_count(fakenewsdataset['title'][y])
C:\Users\LEESEU~1\AppData\Local\Temp/ipykernel_20036/3533526121.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fakenewsdataset["tokenized"][x] = tokenizer.tokenize(y)
C:\Users\Lee Seung Soo\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

,Unnamed: 0,title,text,label,title count,tokenized
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,5,"[You, Can, Smell, Hillary, s, Fear]"
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,14,"[Watch, The, Exact, Moment, Paul, Ryan, Commit..."
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,9,"[Kerry, go, Paris, gesture, sympathy]"
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,15,"[Bernie, supporters, Twitter, erupt, anger, DN..."
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,9,"[The, Battle, New, York, Why, This, Primary, M..."
...,...,...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL,11,"[State, Department, says, can, t, find, emails..."
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE,10,"[The, P, PBS, Should, Stand, Plutocratic, Pent..."
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE,13,"[Anti, Trump, Protesters, Are, Tools, Oligarch..."
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL,11,"[In, Ethiopia, Obama, seeks, progress, peace, ..."


## Splitting data into two Real and Fake dataframes for easier comparison 

In [5]:
#Create dataframe with only fake data
fake = fakenewsdataset[fakenewsdataset['label'] == 'FAKE']
fake.head()

,Unnamed: 0,title,text,label,title count,tokenized
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,5,"[You, Can, Smell, Hillary, s, Fear]"
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,14,"[Watch, The, Exact, Moment, Paul, Ryan, Commit..."
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,15,"[Bernie, supporters, Twitter, erupt, anger, DN..."
5,6903,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",FAKE,2,"[Tehran, USA]"
6,7341,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",FAKE,13,"[Girl, Horrified, At, What, She, Watches, Boyf..."


In [6]:
#Create dataframe with only real data
real = fakenewsdataset[fakenewsdataset['label'] == 'REAL']
real.head()

,Unnamed: 0,title,text,label,title count,tokenized
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,9,"[Kerry, go, Paris, gesture, sympathy]"
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,9,"[The, Battle, New, York, Why, This, Primary, M..."
7,95,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,REAL,5,"[Britain, s, Schindler, Dies, 106]"
8,4869,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,REAL,9,"[Fact, check, Trump, Clinton, commander, in, c..."
9,2909,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,REAL,11,"[Iran, reportedly, makes, new, push, uranium, ..."
